In [1]:
# Check config files

- Check tables match in config and etl_config
- Check subset names match in config and etl_config
- Check dp_names match bo dp_names
- Check scripts in etl_config exists
- Check all stage tables have aliases
- Check aliases match stage table columns
- Check load tables match table and subset names
- Check load scripts exists
- Check table name with source_table
- Check source_code from table match source_code from query

SyntaxError: invalid syntax (<ipython-input-1-138a97a92d2d>, line 3)

In [ ]:
import os
import glob
import pandas as pd

os.chdir('Z:/OMOP/omop_etl/omop_etl/templates')
print(os.getcwd())

from omop_etl.datastore import DataStore
from omop_etl.bo import bo_query
from omop_etl.config import ProjectConfig, ETLConfig

Z:\OMOP\omop_etl\omop_etl\templates


In [ ]:
database = 'dws_omop'
server = 'edw.shands.ufl.edu'

omop = DataStore('config.yml')
omop.connection_str = f'mssql+pyodbc://{server}/{database}?driver=SQL+Server'

with omop.engine.connect() as con:
    bo_queries = bo_query('omop', con)

_config = ProjectConfig('config.yml')
etl_config = ETLConfig()
sql_scripts = [os.path.split(p)[1] for p in glob.glob('Z:/OMOP/omop_etl/omop_etl/sql/*.sql') ]

path = 'Z:/OMOP/omop_etl/omop_etl/sql'

In [ ]:
def file_exists(file):
    path = 'Z:/OMOP/omop_etl/omop_etl/sql'
    filepath = os.path.join(path, file)
    
    return os.path.exists(filepath)

In [ ]:
dp_names = []
preload_scripts = []
mapping_scripts = []
load_scripts = []

for table in config_load.keys():
    if config_load[table]:
        for sbs in config_load[table]:
            stg = etl_stage[table]
            dp_names.append(etl_stage[table][sbs] if isinstance(stg, dict) else stg)
            load_scripts.append(etl_load[table][sbs] if isinstance(stg, dict) else stg)
            
            if isinstance(stg, dict):
                print(f'{table}, {sbs} load script exists?', file_exists(etl_load[table][sbs]))
            
            if table in etl_preload.keys():
                preload_scripts.append(etl_preload[table][sbs])
                print(f'{table}, {sbs} preload script exists?', file_exists(etl_preload[table][sbs]))
                
    else:
        if table in etl_stage.keys():
            dp_names.append(etl_stage[table])
        if table in etl_mapping.keys():
            mapping_scripts.append(etl_mapping[table])
            print(f'{table} mapping script exists?', file_exists(etl_mapping[table]))

person mapping script exists? True
visit_occurrence mapping script exists? True
procedure_occurrence, icd load script exists? True
procedure_occurrence, icd preload script exists? True
procedure_occurrence, cpt load script exists? True
procedure_occurrence, cpt preload script exists? True
drug_exposure, order load script exists? True
drug_exposure, order preload script exists? True
drug_exposure, admin load script exists? True
drug_exposure, admin preload script exists? True
measurement, bp load script exists? True
measurement, bp preload script exists? True
measurement, bp_artline load script exists? True
measurement, bp_artline preload script exists? True
measurement, bp_noninvasive load script exists? True
measurement, bp_noninvasive preload script exists? True
measurement, bp_cvp load script exists? True
measurement, bp_cvp preload script exists? True
measurement, bp_cvp_mean load script exists? True
measurement, bp_cvp_mean preload script exists? True
measurement, bp_map_artline l

In [ ]:
set(dp_names) == set(bo_queries.keys())

False

In [ ]:
# Wrong dp_name in etl_config
set(dp_names) - set(bo_queries.keys())

set()

In [ ]:
# dp_names missing in etl_config
set(bo_queries.keys()) - set(dp_names)

{'MEASUREMENT_Res_RESP_AdultMech', 'MEASUREMENT_Res_RESP_PedsMech'}

In [ ]:
# Check if preload tables exist in config, etl stage, and etl load
for prl in etl_preload:
    if prl in config_load.keys():
        print(f'Preload table {prl} found in config.')
    if prl in etl_stage.keys():
        print(f'Preload table {prl} found in etl_stage')
    if prl in etl_load.keys():
        print(f'Preload table {prl} found in etl_load')

Preload table condition_occurrence found in config.
Preload table condition_occurrence found in etl_stage
Preload table condition_occurrence found in etl_load
Preload table drug_exposure found in config.
Preload table drug_exposure found in etl_stage
Preload table drug_exposure found in etl_load
Preload table measurement found in config.
Preload table measurement found in etl_stage
Preload table measurement found in etl_load
Preload table observation found in config.
Preload table observation found in etl_stage
Preload table observation found in etl_load
Preload table procedure_occurrence found in config.
Preload table procedure_occurrence found in etl_stage
Preload table procedure_occurrence found in etl_load


In [ ]:
temp_path = 'Z:/OMOP/omop_etl/omop_etl/templates'
source = pd.read_csv(os.path.join(temp_path, 'source_to_concept_map.csv'), sep='\t')